In [1]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
directory = "TImages/"

In [3]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [4]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

In [5]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 3:  # Generate 3 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [6]:
create_training_data()

In [7]:
lenofimage = len(training_data)
print(lenofimage)

3900


In [8]:
# Shuffle the training data to ensure randomness
np.random.shuffle(training_data)

In [9]:
# Separate images (X) and labels (y)
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [10]:
# Print the shapes to verify
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (3900, 244, 244, 3)
Shape of y: (3900,)


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_train_reshaped = X_train.reshape(len(X_train), -1)  # Flatten each image
X_test_reshaped = X_test.reshape(len(X_test), -1)
X_train_reshaped.shape

(3120, 178608)

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [15]:
# Feature Scaling
sc = StandardScaler()

X_train_fs = sc.fit_transform(X_train_reshaped)
X_test_fs = sc.transform(X_test_reshaped)


In [16]:
## NAIVE BAYES

In [17]:
from sklearn.naive_bayes import GaussianNB

In [18]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'var_smoothing': [1e-5] # You can adjust this range
    
}

In [19]:
nb = GaussianNB()

In [20]:
# Create the GridSearch object without cross-validation
grid_search_nb = GridSearchCV(nb, param_grid, cv=5, verbose=2)  # Set cv=None for no cross-validation

In [21]:
grid_search_nb.class_prior_ = [0.5, 0.5]

In [22]:
# Fit the GridSearch object to your data
grid_search_nb_train = grid_search_nb.fit(X_train_fs, y_train)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................................var_smoothing=1e-05; total time=   7.4s
[CV] END ................................var_smoothing=1e-05; total time=   7.3s
[CV] END ................................var_smoothing=1e-05; total time=   7.4s
[CV] END ................................var_smoothing=1e-05; total time=   7.2s
[CV] END ................................var_smoothing=1e-05; total time=   7.4s


In [23]:
# Get the best estimator and evaluate it
best_nb = grid_search_nb.best_estimator_
y_test_pred_nb = best_nb.predict(X_test_fs)
accuracy_nb = accuracy_score(y_test, y_test_pred_nb)
print("Best Gaussian Naive Bayes Parameters:", grid_search_nb.best_params_)
print("Accuracy: {:.2f}%".format(accuracy_nb * 100))

Best Gaussian Naive Bayes Parameters: {'var_smoothing': 1e-05}
Accuracy: 26.67%


In [24]:
## for SVM

In [25]:
from sklearn.svm import SVC

In [26]:
# Hyperparameter Tuning using GridSearchCV
param_grid_svm = {'C': [5], 'gamma': ['auto'], 'kernel': ['rbf']}

In [27]:
# Train the SVM model
svm = SVC()
grid_search_svm = GridSearchCV(svm, param_grid_svm, cv=5, verbose=2)
grid_search_svm_train = grid_search_svm.fit(X_train_fs, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 9.4min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 9.2min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time=14.1min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 9.5min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 9.1min


In [28]:
# # Get the best estimator and evaluate it
best_svm = grid_search_svm.best_estimator_
y_test_pred_svm = best_svm.predict(X_test_fs)
accuracy_svm = accuracy_score(y_test, y_test_pred_svm)
print("Best SVM Parameters:", grid_search_svm.best_params_)
print("Accuracy: {:.2f}%".format(accuracy_svm * 100))

Best SVM Parameters: {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
Accuracy: 95.90%


In [29]:
## for KNN

In [30]:
from sklearn.neighbors import KNeighborsClassifier

In [31]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_neighbors': [3, 5]  # You can adjust this range
}

In [32]:
# Create the KNN model
knn = KNeighborsClassifier(n_neighbors = 5, weights='uniform', p = 2, algorithm='auto') # metric = 'minkowski',

In [33]:
# Create the GridSearchCV object
grid_search_knn = GridSearchCV(knn, param_grid, cv=5, verbose=2)

# Fit the GridSearchCV object to your data
grid_search_knn_train = grid_search_knn.fit(X_train_fs, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] END ......................................n_neighbors=3; total time=   8.0s
[CV] END ......................................n_neighbors=3; total time=   8.1s
[CV] END ......................................n_neighbors=3; total time=   8.2s
[CV] END ......................................n_neighbors=3; total time=   8.2s
[CV] END ......................................n_neighbors=3; total time=   7.6s
[CV] END ......................................n_neighbors=5; total time=   7.8s
[CV] END ......................................n_neighbors=5; total time=   7.8s
[CV] END ......................................n_neighbors=5; total time=   8.1s
[CV] END ......................................n_neighbors=5; total time=   8.4s
[CV] END ......................................n_neighbors=5; total time=   8.3s


In [34]:
# Get the best estimator and evaluate it
best_knn = grid_search_knn.best_estimator_
y_test_pred_knn = best_knn.predict(X_test_fs)
accuracy_knn = accuracy_score(y_test, y_test_pred_knn)
print("Best KNN Parameters:", grid_search_knn.best_params_)
print("Accuracy: {:.2f}%".format(accuracy_knn * 100))

Best KNN Parameters: {'n_neighbors': 5}
Accuracy: 95.13%


In [35]:
# GRADIENT BOOSTING

In [36]:
from sklearn.ensemble import GradientBoostingClassifier

In [37]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {'n_estimators': [150],
              'learning_rate': [0.1],
              'max_depth': [5],
              'max_features': ['log2'],
              'loss': ['log_loss'],
              'subsample': [1]
            }

In [38]:
gb = GradientBoostingClassifier()

In [42]:
# Create the GridSearch object without cross-validation
grid_search_gb = GridSearchCV(gb, param_grid, cv=5, verbose=2, error_score='raise')  # Set cv=None for no cross-validation

# Fit the GridSearch object to your data
grid_search_gb_train = grid_search_gb.fit(X_train_fs, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=log2, n_estimators=150, subsample=1; total time=   8.5s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=log2, n_estimators=150, subsample=1; total time=   8.3s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=log2, n_estimators=150, subsample=1; total time=   9.0s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=log2, n_estimators=150, subsample=1; total time=   9.2s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=log2, n_estimators=150, subsample=1; total time=   8.7s


In [43]:
# Get the best estimator and evaluate it
best_gb = grid_search_gb.best_estimator_
y_test_pred_gb = best_gb.predict(X_test_fs)
accuracy_gb = accuracy_score(y_test, y_test_pred_gb)
print("Best Gradient Boosting Parameters:", grid_search_gb.best_params_)
print("Accuracy: {:.2f}%".format(accuracy_gb * 100))

Best Gradient Boosting Parameters: {'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 150, 'subsample': 1}
Accuracy: 97.05%


In [44]:
## RANDOM FOREST

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
param_grid = {
    'n_estimators': [100],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['log2'],
    'bootstrap': [False]
}


In [47]:
# Create the Random Forest model
rf = RandomForestClassifier()

In [48]:
# Create the GridSearchCV object
grid_search_rf = GridSearchCV(rf, param_grid, cv=5, verbose=2, error_score='raise')

In [49]:
# Fit the GridSearchCV object to your data
grid_search_rf_train = grid_search_rf.fit(X_train_fs, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   9.0s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.7s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  12.6s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  12.3s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   9.1s


In [50]:
# Get the best estimator and evaluate it
best_rf = grid_search_rf.best_estimator_
y_test_pred_rf = best_rf.predict(X_test_fs)
accuracy_rf = accuracy_score(y_test, y_test_pred_rf)
print("Best Random Forest Parameters:", grid_search_rf.best_params_)
print("Accuracy: {:.2f}%".format(accuracy_rf * 100))

Best Random Forest Parameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 96.67%


In [54]:
train_list = [grid_search_nb_train.best_score_, 
              grid_search_svm_train.best_score_,
              grid_search_knn_train.best_score_,
              grid_search_gb_train.best_score_,
              grid_search_rf_train.best_score_,            
             ]
test_list = [accuracy_nb, accuracy_svm, accuracy_knn, accuracy_gb, accuracy_rf]
names = ['Naive Bayes', 'SVM', 'KNN', 'Gradient Boosting', 'Random Forest']

train_list = pd.DataFrame(train_list)
test_list = pd.DataFrame(test_list)
names = pd.DataFrame(names)

train_list.columns = ['train']
test_list.columns = ['test']
names.columns = ['names']


over_all_score = pd.concat([names, train_list, test_list], axis=1)
over_all_score

,names,train,test
0,Naive Bayes,0.289103,0.266667
1,SVM,0.951603,0.958974
2,KNN,0.940705,0.951282
3,Gradient Boosting,0.960256,0.970513
4,Random Forest,0.959936,0.966667


In [57]:
pip install plotly


     ---------------------------------------- 15.6/15.6 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 5.1.5
    Uninstalling tenacity-5.1.5:
      Successfully uninstalled tenacity-5.1.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mega-py 1.0.8 requires tenacity<6.0.0,>=5.1.5, but you have tenacity 8.2.3 which is incompatible.


In [68]:
import plotly.graph_objects as go

fig = go.Figure(data=[
#     go.Bar(name='Train Results', x=over_all_score['names'], y=over_all_score['train'],
#            marker=dict(line=dict(width=5)),
#            texttemplate='%{y:20,.2f}', textposition='outside',),

    go.Bar(name='Test Results', x=over_all_score['names'], y=over_all_score['test'],
           marker=dict(line=dict(width=5)),
           texttemplate='%{y:,.2f}', textposition='inside',)
])

fig.update_layout(barmode='group', xaxis={'categoryorder': 'total descending'})

fig.update_xaxes(title_text="Models", title_font={"size": 16})
fig.update_yaxes(title_text="Model Accuracy", title_font={"size": 16})

fig.update_layout(title_text='Train and Test Results of the each Model',
                  title_x=0.5, title_font=dict(size=20))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))

fig.show()